In [45]:
import numpy as np
import pandas as pd
import plotly.io as pio
pio.templates.default = "simple_white"

In [46]:
# load the data
data = pd.read_csv("C:\\Users\omerl\OneDrive\Desktop\iml_repo\IML.HUJI\datasets\\agoda_cancellation_train.csv",engine='python')
data_frame = pd.DataFrame(data)

In [47]:
# get dummies for the categorical charge options
data_frame['charge_option'] = data_frame['charge_option'].apply(lambda x: 1 if x == 'Pay Now' else 0)

In [48]:
# fill missing values at cancellation_datetime column with zero values
# data_frame['cancellation_datetime'] = data_frame['cancellation_datetime'].fillna(0)
# data_frame['cancellation_datetime'] = (
#     pd.to_numeric(data_frame['cancellation_datetime'],errors='coerce').fillna(1))
# data_frame=data_frame.rename(columns = {'cancellation_datetime':'is_cancelled'})


In [49]:
# cleaning the data remove missing values and duplicates
data_frame['cancellation_datetime'] = data_frame['cancellation_datetime'].fillna('2022-02-01')
data_frame['cancellation_datetime'] = pd.to_datetime(data_frame['cancellation_datetime'])
filtered_df = data_frame.loc[(data_frame['cancellation_datetime'] <= '2018-09-29') | (data_frame['cancellation_datetime'] == '2022-02-01')]
filtered_df['booking_datetime'] = filtered_df['booking_datetime'].apply(pd.to_datetime)
filtered_df['is_cancelled'] = (filtered_df['cancellation_datetime'] - filtered_df['booking_datetime']).dt.days
filtered_df['is_cancelled'] = filtered_df['is_cancelled'].apply(lambda x: 1 if x<=44 and x >=35 else 0)
print(filtered_df.shape)
data_frame = filtered_df

(58588, 40)


<ipython-input-49-1ec9945cdcd0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['booking_datetime'] = filtered_df['booking_datetime'].apply(pd.to_datetime)
<ipython-input-49-1ec9945cdcd0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['is_cancelled'] = (filtered_df['cancellation_datetime'] - filtered_df['booking_datetime']).dt.days
<ipython-input-49-1ec9945cdcd0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [50]:
# drop columns that are not relevant to the prediction
data_frame = data_frame.drop(['h_customer_id', 'h_booking_id', 'hotel_area_code'], axis=1)
data_frame = data_frame.drop(['original_payment_method', 'language','original_payment_currency'], axis=1)

In [51]:
# fill missing values with 0 at the following columns
data_frame['request_nonesmoke'] = data_frame['request_nonesmoke'].fillna(0)
data_frame['request_earlycheckin'] = data_frame['request_earlycheckin'].fillna(0)
data_frame['request_latecheckin'] = data_frame['request_latecheckin'].fillna(0)
data_frame['request_twinbeds'] = data_frame['request_twinbeds'].fillna(0)
data_frame['request_largebed'] = data_frame['request_largebed'].fillna(0)
data_frame['request_airport'] = data_frame['request_airport'].fillna(0)
data_frame['request_highfloor'] = data_frame['request_highfloor'].fillna(0)
data_frame['no_of_extra_bed'] = data_frame['no_of_extra_bed'].apply(lambda x: 1 if x>=1 else 0)
data_frame = data_frame.rename(columns = {'no_of_extra_bed':'request_extra_bed'})
### merge all request_* columns into one column
data_frame['request_twinbeds'] = data_frame['request_largebed'] + data_frame['request_twinbeds'] + data_frame['request_airport'] + data_frame['request_highfloor'] + data_frame['request_earlycheckin'] + data_frame['request_latecheckin'] + data_frame['request_nonesmoke']
# rename the request_twinbeds column to number_of_requests
data_frame = data_frame.rename(columns = {'request_twinbeds':'number_of_requests'})
data_frame['number_of_requests'] = data_frame['number_of_requests'].apply(lambda x: 0 if x < 1 else 1)
data_frame.drop(['request_largebed', 'request_airport', 'request_highfloor', 'request_earlycheckin', 'request_latecheckin', 'request_nonesmoke'], axis=1, inplace=True)
# print the correlation between the features and the is_cancelled column

In [52]:
delete_columns = []
dummy_2 = pd.get_dummies(data_frame['cancellation_policy_code'],prefix="cancel_policy")
dummy_2['is_cancelled'] = data_frame['is_cancelled']
print(dummy_2.corr()['is_cancelled'].sort_values(ascending=False)[:80])
print(dummy_2.corr()['is_cancelled'].sort_values(ascending=True)[:20])
for col in dummy_2:
    if(dummy_2[col].sum() < 200):
        dummy_2.drop(col, inplace=True, axis=1)
# for col in dummy_2:
#     if(col == 'is_cancelled'):
#         continue
#     if(np.abs(dummy_2['is_cancelled'].corr(dummy_2[col])) < 0.005):
#         dummy_2.drop(col, inplace=True, axis=1)
#         delete_columns.append(col)
# print(delete_columns)
#data_frame = pd.concat([data_frame, dummy], axis=1)
# print the correlation between the features and the is_cancelled column
dummy_2.drop('is_cancelled', inplace=True, axis=1)
# concat the dummies to the data_frame
data_frame = pd.concat([data_frame, dummy_2], axis=1)

is_cancelled                         1.000000
cancel_policy_8D13P_4D100P_100P      0.045697
cancel_policy_6D2P_5D100P_100P       0.045697
cancel_policy_3D50P_1N               0.040373
cancel_policy_88D100P_100P           0.032049
                                       ...   
cancel_policy_30D90P_90P            -0.000374
cancel_policy_14D21P_10D81P_100P    -0.000374
cancel_policy_90D90P_90P            -0.000374
cancel_policy_13D21P_10D100P_100P   -0.000374
cancel_policy_9D51P_7D100P_100P     -0.000374
Name: is_cancelled, Length: 80, dtype: float64
cancel_policy_365D100P_100P         -0.048072
cancel_policy_7D1N_1N               -0.007420
cancel_policy_4D100P_100P           -0.007168
cancel_policy_3D1N_1N               -0.005526
cancel_policy_7D1N_3D100P_100P      -0.004921
cancel_policy_30D100P_100P          -0.004761
cancel_policy_7D1N_100P             -0.004253
cancel_policy_14D1N_7D100P_100P     -0.003939
cancel_policy_14D50P_7D100P_100P    -0.003849
cancel_policy_6D100P             

In [53]:

# # fill missing values of the hotel_chain_code column with "Unknown"
# data_frame['hotel_chain_code'] = data_frame['hotel_chain_code'].fillna("Unknown")
# # fill missing values of the hotel_brand_code column with "Unknown"
# data_frame['hotel_brand_code'] = data_frame['hotel_brand_code'].fillna("Unknown")
# dummy = pd.get_dummies(data_frame['hotel_chain_code'], prefix='hbc_')
# for col in dummy:
#     if(dummy[col].sum() < 200):
#         dummy.drop(col, inplace=True, axis=1)
# # conect the dummies to the data frame
# data_frame = pd.concat([data_frame, dummy], axis=1)
# # print the correlation between the features and the is_cancelled column
# data_frame.corr()['is_cancelled'].sort_values(ascending=False)

In [54]:
# explore the data in the Booking_datetime column
# and the checkin_date and checkout_date columns
# calculate the difference in days between the Booking_datetime and the checkin_date
# and the difference in days between the Booking_datetime and the checkout_date

data_frame['checkin_date'] = data_frame['checkin_date'].apply(pd.to_datetime)
data_frame['checkout_date'] = data_frame['checkout_date'].apply(pd.to_datetime)
data_frame['booking_datetime'] = data_frame['booking_datetime'].apply(pd.to_datetime)
# insert new column with the difference in days between the Booking_datetime and the checkin_date
data_frame['Lead_day'] = (data_frame['checkin_date'] - data_frame['booking_datetime']).dt.days
data_frame['stay_len'] = (data_frame['checkout_date'] - data_frame['checkin_date']).dt.days
# replace all the negative values with 0
data_frame['Lead_day'] = data_frame['Lead_day'].apply(lambda x: 0 if x < 0 else x)
data_frame['stay_len'] = data_frame['stay_len'].apply(lambda x: 0 if x < 0 else x)

In [55]:
data_frame['original_payment_type'] = data_frame['original_payment_type'].apply(lambda x: 0 if x == 'Credit Card' else 1)

In [56]:
dummy = pd.get_dummies(data_frame['origin_country_code'], prefix="origin_country_code")
dummy.insert(0, 'is_cancelled',data_frame['is_cancelled'])
dummy.fillna("none")
print(dummy.corr()['is_cancelled'].sort_values(ascending=False)[:80])
print(dummy.corr()['is_cancelled'].sort_values(ascending=True)[:20])
counter = 0
for col in dummy:
    if(dummy[col].sum() < 200):
        dummy.drop(col, inplace=True, axis=1)
        delete_columns.append(col)
# for col in dummy:
#     if(col == 'is_cancelled'):
#         continue
#     if(np.abs(dummy['is_cancelled'].corr(dummy[col])) < 0.005):
#         dummy.drop(col, inplace=True, axis=1)
#         delete_columns.append(col)
# print(delete_columns)
dummy.drop('is_cancelled', inplace=True, axis=1)
data_frame = pd.concat([data_frame, dummy], axis=1)

is_cancelled              1.000000
origin_country_code_MT    0.052552
origin_country_code_MO    0.051798
origin_country_code_DZ    0.040774
origin_country_code_AO    0.036990
                            ...   
origin_country_code_MZ   -0.000647
origin_country_code_IS   -0.000647
origin_country_code_CD   -0.000747
origin_country_code_JE   -0.000747
origin_country_code_CL   -0.000747
Name: is_cancelled, Length: 80, dtype: float64
origin_country_code_MY   -0.019134
origin_country_code_TH   -0.017482
origin_country_code_ID   -0.009849
origin_country_code_SA   -0.006008
origin_country_code_RU   -0.005895
origin_country_code_ZA   -0.004686
origin_country_code_KR   -0.004684
origin_country_code_VN   -0.004393
origin_country_code_MM   -0.004361
origin_country_code_LK   -0.004312
origin_country_code_LA   -0.003585
origin_country_code_DE   -0.003497
origin_country_code_AE   -0.003228
origin_country_code_AT   -0.003149
origin_country_code_CN   -0.002967
origin_country_code_NZ   -0.002909
origin_c

In [57]:
dummy_2 = pd.get_dummies(data_frame['guest_nationality_country_name'], prefix="guest_national")
dummy_2['is_cancelled'] = data_frame['is_cancelled']
# print(dummy_2.corr()['is_cancelled'].sort_values(ascending=False)[:80])
# print(dummy_2.corr()['is_cancelled'].sort_values(ascending=True)[:20])
for col in dummy_2:
    if(dummy_2[col].sum() < 100):
        dummy_2.drop(col, inplace=True, axis=1)
        delete_columns.append(col)
# for col in dummy_2:
#     if(col == 'is_cancelled'):
#         continue
#     if(np.abs(dummy_2['is_cancelled'].corr(dummy_2[col])) < 0.005):
#         dummy_2.drop(col, inplace=True, axis=1)
#         delete_columns.append(col)
#data_frame = pd.concat([data_frame, dummy], axis=1)
# print the correlation between the features and the is_cancelled column
dummy_2.drop('is_cancelled', inplace=True, axis=1)

In [58]:
# concat the dummies to the data frame
data_frame = pd.concat([data_frame, dummy_2], axis=1)
# data_frame.drop(['guest_nationality_country_name', 'customer_natio'], axis=1, inlpace=True)

In [59]:
print(data_frame.columns)
data_frame.drop(['cancellation_policy_code', 'checkout_date', 'origin_country_code','guest_nationality_country_name','hotel_id','hotel_live_date', 'hotel_country_code','hotel_live_date','customer_nationality', 'guest_nationality_country_name',
                 'origin_country_code'], axis=1, inplace=True)

Index(['booking_datetime', 'checkin_date', 'checkout_date', 'hotel_id',
       'hotel_country_code', 'hotel_live_date', 'hotel_star_rating',
       'accommadation_type_name', 'charge_option', 'customer_nationality',
       ...
       'guest_national_Spain', 'guest_national_Sweden',
       'guest_national_Switzerland', 'guest_national_Taiwan',
       'guest_national_Thailand', 'guest_national_Turkey',
       'guest_national_United Arab Emirates', 'guest_national_United Kingdom',
       'guest_national_United States', 'guest_national_Vietnam'],
      dtype='object', length=127)


In [60]:
dummy_3 = pd.get_dummies(data_frame['accommadation_type_name'], prefix="type_hotel")
dummy_3['is_cancelled'] = data_frame['is_cancelled']
print(dummy_3.columns)
for col in dummy_3:
    if(dummy_3[col].sum() < 100):
        dummy_3.drop(col, inplace=True, axis=1)
        delete_columns.append(col)
#data_frame = pd.concat([data_frame, dummy], axis=1)
# print the correlation between the features and the is_cancelled column
dummy_3.drop('is_cancelled', inplace=True, axis=1)


Index(['type_hotel_Apartment', 'type_hotel_Boat / Cruise',
       'type_hotel_Bungalow', 'type_hotel_Capsule Hotel', 'type_hotel_Chalet',
       'type_hotel_Guest House / Bed & Breakfast',
       'type_hotel_Holiday Park / Caravan Park', 'type_hotel_Home',
       'type_hotel_Homestay', 'type_hotel_Hostel', 'type_hotel_Hotel',
       'type_hotel_Inn', 'type_hotel_Lodge', 'type_hotel_Love Hotel',
       'type_hotel_Motel', 'type_hotel_Private Villa', 'type_hotel_Resort',
       'type_hotel_Resort Villa', 'type_hotel_Ryokan',
       'type_hotel_Serviced Apartment', 'type_hotel_Tent',
       'type_hotel_UNKNOWN', 'is_cancelled'],
      dtype='object')


In [61]:
data_frame = pd.concat([data_frame, dummy_3], axis=1)
# print the correlation between the features and the is_cancelled column
print(data_frame.columns)
print(data_frame.info())
print(data_frame.isnull().sum())
print(data_frame.shape)


Index(['booking_datetime', 'checkin_date', 'hotel_star_rating',
       'accommadation_type_name', 'charge_option', 'guest_is_not_the_customer',
       'no_of_adults', 'no_of_children', 'request_extra_bed', 'no_of_room',
       ...
       'type_hotel_Capsule Hotel', 'type_hotel_Guest House / Bed & Breakfast',
       'type_hotel_Home', 'type_hotel_Hostel', 'type_hotel_Hotel',
       'type_hotel_Motel', 'type_hotel_Resort', 'type_hotel_Resort Villa',
       'type_hotel_Ryokan', 'type_hotel_Serviced Apartment'],
      dtype='object', length=131)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 58588 entries, 0 to 58658
Columns: 131 entries, booking_datetime to type_hotel_Serviced Apartment
dtypes: bool(2), datetime64[ns](3), float64(4), int64(12), object(1), uint8(109)
memory usage: 15.6+ MB
None
booking_datetime                 0
checkin_date                     0
hotel_star_rating                0
accommadation_type_name          0
charge_option                    0
                    

In [62]:
data_frame.drop(['accommadation_type_name'], axis=1, inplace=True)

In [64]:
# create a new csv file with the new data frame
# data_frame.to_csv('new_data_frame.csv', index=False)

In [65]:
print(data_frame.columns)
print(data_frame.info())
print(data_frame.isnull().sum())
print(data_frame.shape)

Index(['booking_datetime', 'checkin_date', 'hotel_star_rating',
       'charge_option', 'guest_is_not_the_customer', 'no_of_adults',
       'no_of_children', 'request_extra_bed', 'no_of_room',
       'original_selling_amount',
       ...
       'type_hotel_Capsule Hotel', 'type_hotel_Guest House / Bed & Breakfast',
       'type_hotel_Home', 'type_hotel_Hostel', 'type_hotel_Hotel',
       'type_hotel_Motel', 'type_hotel_Resort', 'type_hotel_Resort Villa',
       'type_hotel_Ryokan', 'type_hotel_Serviced Apartment'],
      dtype='object', length=130)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 58588 entries, 0 to 58658
Columns: 130 entries, booking_datetime to type_hotel_Serviced Apartment
dtypes: bool(2), datetime64[ns](3), float64(4), int64(12), uint8(109)
memory usage: 15.1 MB
None
booking_datetime                 0
checkin_date                     0
hotel_star_rating                0
charge_option                    0
guest_is_not_the_customer        0
                         